In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('diabetes.csv')

In [4]:
from sklearn.impute import SimpleImputer

In [5]:
si = SimpleImputer(strategy='mean', missing_values=0)

In [6]:
impute_features = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[impute_features] = si.fit_transform(df[impute_features])

In [7]:
dataset = df.to_numpy()

In [8]:
dataset_x = dataset[:, :-1]
dataset_y = dataset[:, -1]

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
training_dataset_x, test_dataset_x, training_dataset_y, test_dataset_y = train_test_split(dataset_x, dataset_y, test_size=0.2)

In [11]:
from tensorflow.keras.callbacks import Callback

In [12]:
class MyCallback(Callback):
    def on_epoch_end(self, epoch, logs):
        loss = logs['loss']
        val_loss = logs['val_loss']
        print(f'epoch: {epoch}, loss: {loss}, val_loss: {val_loss}')

In [13]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense

In [14]:
model = Sequential(name='Diabetes')

In [ ]:
model.add(Input((training_dataset_x.shape[1],)))
model.add(Dense(16, activation='relu', name='Hidden-1'))
model.add(Dense(16, activation='relu', name='Hidden-2'))
model.add(Dense(1, activation='sigmoid', name='Output'))
model.summary()

In [16]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_accuracy'])

In [17]:
mycallback = MyCallback()

In [ ]:
hist = model.fit(training_dataset_x, training_dataset_y, batch_size=32, epochs=300, validation_split=0.2, callbacks=[mycallback], verbose=0)

In [19]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(14, 6))
plt.title('Epoch - Loss Graph', pad=10, fontsize=14)
plt.xticks(range(0, 300, 10))
plt.plot(hist.epoch, hist.history['loss'])
plt.plot(hist.epoch, hist.history['val_loss'])
plt.legend(['Loss', 'Validation Loss'])
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))
plt.title('Epoch - Binary Accuracy Graph', pad=10, fontsize=14)
plt.xticks(range(0, 300, 10))
plt.plot(hist.epoch, hist.history['binary_accuracy'])
plt.plot(hist.epoch, hist.history['val_binary_accuracy'])
plt.legend(['Accuracy', 'Validation Accuracy'])
plt.show()

In [ ]:
eval_result = model.evaluate(test_dataset_x, test_dataset_y, batch_size=32)

In [ ]:
for i in range(len(eval_result)):
    print(f'{model.metrics_names[i]}: {eval_result[i]}')

In [24]:
def on_epoch_end_proc(epoch, logs):
    pass
def on_batch_begin_proc(batch, logs):
    pass
def on_batch_end_proc(batch, logs):
    pass

from tensorflow.keras.callbacks import LambdaCallback

lambda_callback = LambdaCallback(on_epoch_end=on_epoch_end_proc, on_batch_begin=on_batch_begin_proc, on_batch_end=on_batch_end_proc)
hist = model.fit(training_dataset_x, training_dataset_y, batch_size=32, epochs=300, validation_split=0.2, callbacks=[lambda_callback], verbose=0)